In [1]:
from random import random, sample
import numpy as np

from astropy import coordinates
from astroquery.esa.hubble import ESAHubbleClass
from astropy.io import fits
from astroquery.mast import Observations
from astropy.table import vstack, Table

import matplotlib.pyplot as plt
from PIL import Image
from skimage.transform import resize

import sys
import os
import time
import re
import json
 
import requests
from urllib.parse import quote as urlencode

In [2]:
def process_image(img: np.ndarray):

    img[np.isnan(img)] = 0
    
    RES_DIFF_CONST = 34.5  #TUG_RES / HUBBLE_RES
    
    x_sz = img.shape[0]
    y_sz = img.shape[1]

    x_cal = x_sz / RES_DIFF_CONST
    y_cal = y_sz / RES_DIFF_CONST

    x_lower = (RES_DIFF_CONST * 128) * (x_sz//(RES_DIFF_CONST * 128))
    x_upper = (RES_DIFF_CONST * 128) * ((x_sz//(RES_DIFF_CONST * 128)) + 1)

    print(x_lower)
    print(x_upper)
    
    
    y_lower = (RES_DIFF_CONST * 128) * (y_sz//(RES_DIFF_CONST * 128))
    y_upper = (RES_DIFF_CONST * 128) * ((y_sz//(RES_DIFF_CONST * 128)) + 1)

    x_resize = None
    y_resize = None
    
    if np.abs(x_sz - x_lower) < np.abs(x_sz - x_upper):
        x_resize = x_lower / RES_DIFF_CONST

    else:
        x_resize = x_upper/RES_DIFF_CONST


    if np.abs(y_sz - y_lower) < np.abs(y_sz - y_upper):
        y_resize = y_lower/RES_DIFF_CONST

    else:
        y_resize = y_upper/RES_DIFF_CONST

    img = resize(img, (x_resize, y_resize))

    rows = img.shape[0]
    
    sections = (img
              .reshape(rows // 128, 128, -1, 128)
              .swapaxes(1, 2)
              .reshape(-1, 128, 128))

    return sections